# Resample with xarray:

https://stackoverflow.com/questions/50506347/operation-along-year-with-xarray

In the below link: Organizing daily Excel data into xarray dataset  (There is a nice 3-D figure represents the N-Data array)

https://stackoverflow.com/questions/41904951/organizing-daily-excel-data-into-xarray-dataset

https://medium.com/pangeo/step-by-step-guide-to-building-a-big-data-portal-e262af1c2977

https://stackoverflow.com/questions/40544846/read-multiple-coordinates-with-xarray

http://stephanhoyer.com/2015/06/11/xray-dask-out-of-core-labeled-arrays/


In [143]:
%matplotlib inline
import numpy as np
import xarray as xr
import datetime as dt
import matplotlib.pyplot as plt

In [50]:
###http://stephanhoyer.com/2015/06/11/xray-dask-out-of-core-labeled-arrays/
ds=xr.open_dataset('Hokk_Ens_Aggreg_S1_S2_2017_2018.nc')
ds.sel(time='2017-1-1').max(['latitude', 'longitude'])

<xarray.Dataset>
Dimensions:  (number: 10, time: 8)
Coordinates:
  * number   (number) int32 0 1 2 3 4 5 6 7 8 9
  * time     (time) datetime64[ns] 2017-01-01 ... 2017-01-01T21:00:00
Data variables:
    u10      (time, number) float32 6.455375 13.561745 ... 11.478749 17.049562
    v10      (time, number) float32 14.842304 13.009474 ... 14.130638 1.0267302
    t2m      (time, number) float32 294.8644 302.89862 ... 300.94647 299.64136
    asn      (time, number) float32 0.8447724 0.7709349 ... 0.84997314 0.8614609
    sd       (time, number) float32 0.16576606 0.22333804 ... 0.2209382
    ssrdc    (time, number) float32 8926541.0 9398559.0 ... 7235145.0 7391869.0
    ssrd     (time, number) float32 9821467.0 8115192.0 ... 9900984.0 9901138.0
    tcc      (time, number) float32 0.9629195 0.92849404 ... 0.99993896 0.969481
    tciw     (time, number) float32 0.61522216 0.8791555 ... 0.63662213
    tclw     (time, number) float32 0.9706412 0.94882774 ... 1.4505367 1.2978427
    tp       (ti

In [52]:
# axis labels are (time, latitude, longitude)
ds.t2m.values[[40, 41, 42, 43, 44, 45, 46, 47]].max(axis=(1, 2))
dst2m=ds.t2m

In [61]:
# dst2m.to_netcdf('my-dataset2.nc') ## To save data as netcdf  (nc file)
# saved_ds = xr.open_dataset('my-dataset2.nc')
# assert saved_ds.equals(dst2m)
# saved_ds

Or you might pass your data off to pandas for further analysis in tabular form: https://xray.readthedocs.io/en/stable/pandas.html

In [118]:
import time
import sys
df_t2m= dst2m.to_dataframe();
df_tCdeg = df_t2m-273.15;  #convert temperature from Kelvin to Celsius
# %time float(df_tCdeg.mean())
# df_tCdeg.head()
# df.describe()
%time float(df_tCdeg.mean())  #calculate time and CPU time


#####%time 9 / 5.0 * (df_t2m - 273.15) + 32; # convert from Kelvin to degrees Fahrenheit

# ds.nbytes * (2 ** -30)

Wall time: 0 ns


4.401648998260498

http://stephanhoyer.com/2015/06/11/xray-dask-out-of-core-labeled-arrays/#fn:1

For distributed computation that lets us automatically parallelize grouped operations written like:

s.groupby('some variable').apply(f),  where f is a function on NumPy arrays.

In [137]:
%%time
# everything is fast until we compute values
ds_by_season = ds.groupby('time.season').mean('time')
t2m_range = abs(ds_by_season.sel(season='JJA')- ds_by_season.sel(season='DJF')).t2m

Wall time: 1.94 s


In [138]:
# now we calculate the actual result
%time result = t2m_range.load()

Wall time: 9.77 ms


In [146]:
####In case of climatic data of temperature from 1979 to 2015
####http://stephanhoyer.com/2015/06/11/xray-dask-out-of-core-labeled-arrays/#fn:1
# from datetime import datetime, timedelta 
# import numpy as np
# plt.figure(figsize=(15, 6))
# plt.imshow(result, cmap='cubehelix', vmin=0, vmax=60,
#            extent=[43, 43, 142, 144], interpolation='nearest')
# plt.title('Summer minus winter mean temperature')
# plt.colorbar(label='degrees C', extend='max');

http://www.matteodefelice.name/post/c3s-multimodel/
Now I can merge the models using the concatenation function from xarray. For this operation you have to specify along which dimension you want to concatenate along the data cubes and, very important, the dimensions must be consistent, otherwise everything will become very messy!